In [ ]:
import numpy as np
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load processed notes
with open("../data/processed_notes.pkl", "rb") as f:
    notes = pickle.load(f)

# Create a mapping of notes to integers
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Convert notes to integer sequences
sequence_length = 50  # Number of notes per input sequence
input_sequences = []
output_notes = []

for i in range(len(notes) - sequence_length):
    input_sequences.append([note_to_int[n] for n in notes[i:i + sequence_length]])
    output_notes.append(note_to_int[notes[i + sequence_length]])

# Reshape for LSTM
X = np.array(input_sequences)
y = to_categorical(output_notes, num_classes=len(unique_notes))

# Pad sequences to ensure uniform length
X = pad_sequences(X, maxlen=sequence_length, padding='pre')

# Build LSTM Model
model = Sequential([
    Embedding(len(unique_notes), 100, input_length=sequence_length),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dense(256, activation='relu'),
    Dense(len(unique_notes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(X, y, epochs=50, batch_size=64, verbose=1)

# Save Model
model.save("../models/melody_generator.h5")

print("✅ Training complete! Model saved to ../models/melody_generator.h5")




Epoch 1/50
